In [ ]:
import csv
import pandas as pd
import numpy as np
import keras

airlines = pd.read_csv('airlines.csv')
airlines.head()


import tensorflow as tf
from tensorflow.python.client import device_lib
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
print(airlines.shape)
airlines.isna().sum() #no na's

In [ ]:
airports = pd.read_csv("airports.csv")
airports.head()

In [ ]:
print(airports.shape)
airports.isna().sum()

In [ ]:
airports #the Kaggle file airports.csv had 3 missing values that we filled in manually after research
#please use airports.csv file that was sent with this code

In [ ]:
test = pd.read_csv("flights_test.csv")
test.head()

In [ ]:
print(test.shape)
test.isna().sum() #no na's

In [ ]:
train = pd.read_csv("flights_train.csv")
train.head()
print(train.shape)
train.isna().sum() #no na's

In [ ]:
train.head()

In [ ]:
submitSample = pd.read_csv("submit_sample.csv")
submitSample.head()

print(submitSample.shape)
submitSample.isna().sum() #no na's


In [ ]:
import matplotlib.pyplot as plt

plt.hist(train["ARRIVAL_DELAY"], density= True, bins=1000)  # density=False would make counts
plt.ylabel('Counts')
plt.xlabel('Delays')
plt.xlim([-100, 400])
plt.show()

#most delays are centered around 0

In [ ]:
#scheduled dep, dep time, scheduled arrival are in HHMM, turn into minutes 
#arrival delay,taxi out, wheels off, and scheduled time (length of flight) are in minutes

train["SCHEDULED_DEPT_MIN"] = ((train["SCHEDULED_DEPARTURE"]/100).apply(np.floor)*60) + train["SCHEDULED_DEPARTURE"].mod(100)

train["DEPT_TIME_MIN"] = ((train["DEPARTURE_TIME"]/100).apply(np.floor)*60) + train["DEPARTURE_TIME"].mod(100)

train["SCHEDULED_ARRIVAL_MIN"] = ((train["SCHEDULED_ARRIVAL"]/100).apply(np.floor)*60) + train["SCHEDULED_ARRIVAL"].mod(100)
 
#drop columns in HHMM
train = train.drop(columns=['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', "SCHEDULED_ARRIVAL"], axis=1)

In [ ]:
#same processing of HHMM columns on test set
test["SCHEDULED_DEPT_MIN"] = ((test["SCHEDULED_DEPARTURE"]/100).apply(np.floor)*60) + test["SCHEDULED_DEPARTURE"].mod(100)

test["DEPT_TIME_MIN"] = ((test["DEPARTURE_TIME"]/100).apply(np.floor)*60) + test["DEPARTURE_TIME"].mod(100)

test["SCHEDULED_ARRIVAL_MIN"] = ((test["SCHEDULED_ARRIVAL"]/100).apply(np.floor)*60) + test["SCHEDULED_ARRIVAL"].mod(100)

test = test.drop(columns=['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', "SCHEDULED_ARRIVAL"])

In [ ]:
# merge train dataset with airports.csv dataset to get latitude and longitude

#for departure airport
train= train.merge(airports[["IATA_CODE", "LONGITUDE", "LATITUDE"]], 
                   left_on= "ORIGIN_AIRPORT", right_on="IATA_CODE", how="left")
train=train.drop(columns=["IATA_CODE"])
train = train.rename(columns={'LONGITUDE': 'ORIGIN_LONGITUDE', 'LATITUDE': 'ORIGIN_LATITUDE'})

#for destination airport
train= train.merge(airports[["IATA_CODE", "LONGITUDE", "LATITUDE"]], 
                   left_on= "DESTINATION_AIRPORT", right_on="IATA_CODE", how="left")
train=train.drop(columns=["IATA_CODE"])
train = train.rename(columns={'LONGITUDE': 'DESTINATION_LONGITUDE', 'LATITUDE': 'DESTINATION_LATITUDE'})



In [ ]:
# merge test dataset with airports.csv dataset to get latitude and longitude

#for departure airport
test= test.merge(airports[["IATA_CODE", "LONGITUDE", "LATITUDE"]], 
                   left_on= "ORIGIN_AIRPORT", right_on="IATA_CODE", how="left")
test=test.drop(columns=["IATA_CODE"])
test = test.rename(columns={'LONGITUDE': 'ORIGIN_LONGITUDE', 'LATITUDE': 'ORIGIN_LATITUDE'})

#for destination airport
test= test.merge(airports[["IATA_CODE", "LONGITUDE", "LATITUDE"]], 
                   left_on= "DESTINATION_AIRPORT", right_on="IATA_CODE", how="left")
test=test.drop(columns=["IATA_CODE"])
test = test.rename(columns={'LONGITUDE': 'DESTINATION_LONGITUDE', 'LATITUDE': 'DESTINATION_LATITUDE'})



In [ ]:
train.isna().sum() #check that there are no nan's

In [ ]:
test.isna().sum() #check that there are no nan's

In [ ]:
#delete id, year since id does not provide useful information and year is 2015 for all rows
train = train.drop('id', 1)
train = train.drop("YEAR", 1)
test = test.drop('id', 1)
test = test.drop("YEAR", 1)

In [ ]:
### trying neural network for regression
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#turn categorical variables into category codes

train['DAY_OF_WEEK']=train['DAY_OF_WEEK'].astype("category").cat.codes
train['FLIGHT_NUMBER']=train['FLIGHT_NUMBER'].astype("category").cat.codes
train['AIRLINE']=train['AIRLINE'].astype("category").cat.codes
train['TAIL_NUMBER']=train['TAIL_NUMBER'].astype("category").cat.codes
train['ORIGIN_AIRPORT']=train['ORIGIN_AIRPORT'].astype("category").cat.codes
train['DESTINATION_AIRPORT']=train['DESTINATION_AIRPORT'].astype("category").cat.codes

In [ ]:
#turn categorical variables into category codes
test['DAY_OF_WEEK']=test['DAY_OF_WEEK'].astype("category").cat.codes
test['FLIGHT_NUMBER']=test['FLIGHT_NUMBER'].astype("category").cat.codes
test['AIRLINE']=test['AIRLINE'].astype("category").cat.codes
test['TAIL_NUMBER']=test['TAIL_NUMBER'].astype("category").cat.codes
test['ORIGIN_AIRPORT']=test['ORIGIN_AIRPORT'].astype("category").cat.codes
test['DESTINATION_AIRPORT']=test['DESTINATION_AIRPORT'].astype("category").cat.codes



In [ ]:
#split features from outcome variable

x1 = train.loc[:, train.columns != "ARRIVAL_DELAY"]
y1 = train.loc[:,"ARRIVAL_DELAY"]

#split train and validation sets
X_train, X_val, y_train, y_val = train_test_split(x1, y1)


In [ ]:
#scale the data

y_train=np.reshape(y_train.ravel(), (-1,1))
y_val=np.reshape(y_val.ravel(), (-1,1))

scaler_x =StandardScaler()
scaler_y =StandardScaler()

print(scaler_x.fit(X_train))
xtrain_scale=scaler_x.transform(X_train)
print(scaler_x.fit(X_val))
xval_scale=scaler_x.transform(X_val)
test_scaled = scaler_x.transform(test) 

print(scaler_y.fit(y_train))
ytrain_scale=scaler_y.transform(y_train)
print(scaler_y.fit(y_val))
yval_scale=scaler_y.transform(y_val)


In [ ]:
from keras import callbacks

#optimizer for epoch number. Argument "patience" determines the number of epochs to continue to try if the loss is not improving before aborting

earlystopping = callbacks.EarlyStopping(monitor = "val_loss",
                                       mode = "min", patience = 10,
                                       restore_best_weights = True)

In [ ]:
from matplotlib import pyplot as plt


In [ ]:
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [ ]:
# define the model

model = Sequential()
model.add(Dense(5000, input_dim=19, activation='relu')) #dim: number of columns
model.add(Dense(1000, activation='relu'))

#Output layer
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mse",'mae'])
model.summary()


In [ ]:
#train the model, use 0.99 of the data for training
history = model.fit(xtrain_scale, ytrain_scale, batch_size=10000, epochs = 100, validation_split=0.01, callbacks =[earlystopping])


In [ ]:
predictions = model.predict(xval_scale)
print(history.history.keys())
# Plot "Loss" per number of epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#accuracy measures on train set
predictions = model.predict(xtrain_scale)
predictions = scaler_y.inverse_transform(predictions)
predictions
print(mean_squared_error(y_train, predictions))
print(math.sqrt(mean_squared_error(y_train, predictions)))
print(np.mean(y_train))
print(np.mean(predictions))

In [ ]:
#accuracy measures on validation set
predictions = model.predict(xval_scale)
predictions = scaler_y.inverse_transform(predictions)
predictions
print(mean_squared_error(y_val, predictions))
print(math.sqrt(mean_squared_error(y_val, predictions)))
print(np.mean(y_val))
print(np.mean(predictions))

In [ ]:
#scaled predictions on test set
predictions = model.predict(test_scaled)
print("Predicted values are: ", predictions)


In [ ]:
#reverse the scaling
predictions = scaler_y.inverse_transform(predictions)
predictions

In [ ]:
#export to file

y_predTest= predictions[:,0]
df = pd.DataFrame(y_predTest)
df.index=[x for x in range(0, len(df.values))]
df.index.name="id"
df.columns = [ "ARRIVAL_DELAY"]
df.head()
df.to_csv("submissionModel.csv")